In [1]:
import bs4
import requests
import re
import sqlite3
import pandas as pd
import datetime

In [2]:
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
days = [31,29,31,30,31,30,31,31,30,31,30,31]
month_mapping ={"January" : "01", 
                "February" : "02",
                "March" : "03",
                "April" : "04",
                "May" : "05",
                "June" : "06",
                "July" : "07",
                "August" : "08",
                "September" : "09",
                "October" : "10",
                "November" : "11",
                "December" : "12"}

In [3]:
urls = []

for index, month in enumerate(months):
    for day in range(days[index]):
        urls.append(str(month) + "_" + str(day+1))

In [4]:
def clean_year(year):
    year = year.replace("AD ", "")
    year = year.replace(" (probable)", "")
    
    if "BC" in year:
        year = year.replace(" BCE", "")
        year = year.replace(" BC", "")
        while len(year) < 4:
            year = "0" + str(year)
        return year, True
    
    while len(year) < 4:
        year = "0" + str(year)
    return year, False

def clean_time(time):
    time = re.sub("\s+", "", time)
    return time

def preprocess_row(row, event, date):
    dict1={}
     # Removes [1] from the end - e.g. They rebel and proclaim Vitellius as emperor.[1]
    row = re.sub("\[[0-9]*\]", "", row.text)
    
    if len(row.split(' – ')) > 1:
        dict1['event'] = event
        year = row.split(' – ')[0]
        year, bc = clean_year(year)
        time = str(year) + "-" + date
        dict1['description'] = row.split(' – ')[1]
        if bc:
            dict1['time'] = None
            dict1['bc'] = clean_time(time)
        else:
            dict1['time'] = clean_time(time) 
            dict1['bc'] = None
        
    elif len(row.split(' - ')) > 1:
        dict1['event'] = event
        year = row.split(' - ')[0]
        year, bc = clean_year(year)
        time = str(year) + "-" + date
        dict1['description'] = row.split(' - ')[1]
        if bc:
            dict1['time'] = None
            dict1['bc'] = clean_time(time)
        else:
            dict1['time'] = clean_time(time)
            dict1['bc'] = None
            
    return dict1

In [6]:
base_url = "https://en.wikipedia.org/wiki/"

ids = ['Births', 'Deaths', 'Events']

rows = []

for url in urls:
    response = requests.get(base_url + url) # retrieve html from each url
    
    if response is not None:
        page = bs4.BeautifulSoup(response.text, 'html.parser')
        
        date = month_mapping[url.split("_")[0]] + "-"
        
        day = str(url.split("_")[1])
        if len(day)<2:
            day = "0"+day
        date += day
        
        for index, element in enumerate(page.find_all(['h2', 'ul'])):
            if element.name == 'h2':
                event_type = list(map(lambda x: x in str(element), ids)) # a list of booleans corresponding to the event type
                if True in event_type: # if Births/Events/Deaths
                    event = ids[event_type.index(True)]
                    content = page.find_all(['h2', 'ul'])[index+1].find_all('li')
                    for row in content:
                        rows.append(preprocess_row(row, event, date))
    else:
        print("Response is None for: ", base_url + url)

In [7]:
df = pd.DataFrame(rows) # populate a dataframe from the scraped data

In [8]:
len(df)

131255

In [9]:
df.isna().sum()

bc             131100
description       295
event             295
time              450
dtype: int64

In [10]:
# drop all rows in which all values are NaN?
len(df.dropna(how='all')) 

130960

In [11]:
df.dropna(how='all',inplace=True)

In [12]:
df.isna().sum() 
# remaining NaN values in 'time' column are the entries which had BC/BCE years
# they have a corresponding value in 'bc' column

bc             130805
description         0
event               0
time              155
dtype: int64

In [14]:
# creates a databse
conn = sqlite3.connect("wikipedia.db")

In [15]:
# populates the database from the dataframe
df.to_sql(name='wikipedia.db', con=conn)

In [16]:
conn.commit()
conn.close()